## Team Virgo Cluster - Lin Cheng
### Team document link:
https://colab.research.google.com/drive/1573CEdUu47ptIx9mxFUGxPBTE7mamCrS

### My progress:
- Starting from last week's work in which we got accuracy 0.9 on the LiarLiar dataset, this week I leverage Hyunwook and Yu's data enrichment source, and tried more approaches to enhance the performance.  
- With the additional 50K non-fake news samples, I can now make all the LiarLiar data "bs" as some types are very few and have close meaning to bs:
"bias            443
bs            11492
conspiracy      430
fake             19
hate            246
junksci         102
satire          146
state           121".  
- Calculate TF-IDF scores and get the vectorised doc-word metrix for further modeling.  
- Add compound feature sentiment.  
- Apply differnt ALGs and turn the models, found the best accuracy 97% with LogisticRegression(class_weight={"bs":5,"non-bs":3}, {'C': 1, 'penalty': 'l1'}), which is a notable improvement. 
- Also played with LDA to see how it works for topic modeling.  



## Datasets
Original Kaggle fake news dataset: 
https://github.com/synle/machine-learning-sample-dataset/raw/master/liar_dataset/kaggle/kaggle-fake.csv

Enriched Kaggle news dataset (50,000 verified non-fake news):
https://github.com/h7shin/all_news_dataset_kaggle/raw/master/articles1.csv

In [2]:
# dependencies
import pandas as pd
import nltk
import numpy as np

# download nltk stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import pandas as pd
import io
import requests

def get_parsed_data2(url):
    return pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')), sep=',', header='infer')

# download and parse the dataset...
data_kg_fake_news = get_parsed_data2('https://github.com/synle/machine-learning-sample-dataset/raw/master/liar_dataset/kaggle/kaggle-fake.csv')

In [0]:
data_kg_nonfake_news = get_parsed_data2('https://github.com/h7shin/all_news_dataset_kaggle/raw/master/articles1.csv')

In [0]:
data_kg_nonfake_news.rename(columns={"content": "text"}, inplace=True)
data_kg_nonfake_news['type'] = 'non-bs'

In [6]:
print(data_kg_nonfake_news.shape)
data_kg_nonfake_news.head()

(50000, 11)


,Unnamed: 0,id,title,publication,author,date,year,month,url,text,type
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,non-bs
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",non-bs
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",non-bs
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",non-bs
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",non-bs


In [7]:
data_kg_fake_news.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias




## Transform data

Combine those two datasets, mark data "bias 443 bs 11492 conspiracy 430 fake 19 hate 246 junksci 102 satire 146 state 121" to "bs".

In [8]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
import numpy as np
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from string import punctuation
from nltk import PorterStemmer
import copy 
import re
from sklearn.model_selection import train_test_split

nltk.download('punkt')

cachedStopWords = set(stopwords.words('english') + list(punctuation) + [''])
print(data_kg_fake_news.shape)
print(data_kg_fake_news.groupby(['type'])['type'].count())

print(data_kg_nonfake_news.shape)
print(data_kg_nonfake_news.groupby(['type'])['type'].count())

data_kg_fake_news_b=copy.deepcopy(data_kg_fake_news);
data_kg_fake_news_b.loc[data_kg_fake_news_b['type']!='non-bs', 'type'] = 'bs'

all_data = pd.concat([data_kg_fake_news_b[['text','type']], data_kg_nonfake_news[['text','type']]])

print(all_data.groupby(['type'])['type'].count())

print(all_data.shape)
X=all_data['text'].astype('U')
y=all_data['type']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
(12999, 20)
type
bias            443
bs            11492
conspiracy      430
fake             19
hate            246
junksci         102
satire          146
state           121
Name: type, dtype: int64
(50000, 11)
type
non-bs    50000
Name: type, dtype: int64
type
bs        12999
non-bs    50000
Name: type, dtype: int64
(62999, 2)


## TF-IDF

Now try to use TfidfVectorizer to get a matrix for further classification. Also tried applying SVD for dimension reduction.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, Imputer, MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

def tokenize2(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords]
    tokens = list(map(lambda token: PorterStemmer().stem(token), words))
    p = re.compile('[a-zA-Z]+')
    filtered_tokens = list(filter(lambda token: p.match(token) and len(token) >= min_length, tokens))
    return filtered_tokens

vectorizer = TfidfVectorizer(tokenizer=tokenize2, max_features=2000)
svd_model = TruncatedSVD(n_components=200,       
                         algorithm='randomized',
                         n_iter=10)
# svd_transformer = Pipeline([('tfidf', vectorizer), 
#                             ('svd', svd_model)])
svd_transformer=vectorizer
    
vectorised_documents = svd_transformer.fit_transform(X)

### Add  Sentiment feature

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

def sentiment(x):
    score = sid.polarity_scores(x)
    return score['compound']
    
sentiment = all_data['text'].astype('U').apply(lambda x : sentiment(x))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
X = pd.concat([sentiment.reset_index(),
                        pd.DataFrame(vectorised_documents.toarray(),
                        columns=vectorizer.vocabulary_)],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Modeling and tuning
- Random forest  
- SVM  
- Logistic Regression

In [21]:
clf=RandomForestClassifier()
clf.fit(X_train, y_train)
feature_imp = pd.Series(clf.feature_importances_,index=list(X)).sort_values(ascending=False).nlargest(20)
print(feature_imp)

y_pred=clf.predict(X_test)
print(y_test.value_counts(sort=False))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


index       0.191349
duti        0.023788
tend        0.018595
appear      0.016418
except      0.013192
quickli     0.012568
ukrain      0.011429
carolina    0.009436
panel       0.008980
cancer      0.008024
scandal     0.007848
obtain      0.007031
defend      0.006779
contact     0.005981
key         0.005947
ms.         0.005886
object      0.005816
remark      0.005783
soon        0.005269
fast        0.004903
dtype: float64
bs         3844
non-bs    15056
Name: type, dtype: int64
Accuracy: 0.9496825396825397
[[ 3153   691]
 [  260 14796]]


In [0]:
#Caution: SLOW
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn import svm

# scaler = MaxAbsScaler()
# vectorised_train_documents = scaler.fit_transform(vectorised_train_documents)

# svc = svm.SVC()
# parameters = {'kernel':('linear','sigmoid')}
# lr_gridCV = GridSearchCV(svc, parameters, cv=5, scoring='accuracy')
# lr_gridCV = lr_gridCV.fit(vectorised_train_documents, y_train)

# print(lr_gridCV.best_score_)
# print(lr_gridCV.best_params_)
# y_pred = lr_gridCV.predict(vectorised_test_documents)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [23]:

# logistic = LogisticRegression()
# logistic = LogisticRegression(class_weight='balanced')
logistic = LogisticRegression(class_weight={"bs":5,"non-bs":3})
C = [0.1, 1]
penalty = ['l1','l2']

param_grid = dict(C=C, penalty=penalty)
gs = GridSearchCV(logistic, param_grid=param_grid, cv= 5, scoring='accuracy')

gs.fit(X_train, y_train)
print(gs.best_params_)

y_pred=gs.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred, pos_label='bs'))
print(metrics.confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

{'C': 1, 'penalty': 'l1'}
Accuracy: 0.9706878306878307
F1: 0.9290289520881373
[[ 3626   218]
 [  336 14720]]


So by trying different algorisms and model tuning, eventually we found weighted LogisticRegression gives pretty good result. And we also learnt data enrichment and data transformation based on proper Interpretation are super crucial for machine learning.

In [38]:
print( all_data['text'].__class__)
print( all_data['text'].head())

<class 'pandas.core.series.Series'>
0    Print They should pay all the back all the mon...
1    Why Did Attorney General Loretta Lynch Plead T...
2    Red State : \nFox News Sunday reported this mo...
3    Email Kayla Mueller was a prisoner and torture...
4    Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...
Name: text, dtype: object


In [0]:
def sentiment(x):
    score = sid.polarity_scores(x)
    return score['compound']

data = np.array(["Had President Donald Trump been successful in launching prosecutions against Hillary Clinton and James Comey, it could have spelled the end of his presidency, as a clear-cut abuse of power.\
It never happened, apparently thwarted by then-White House Counsel Don McGahn and other senior officials. But that does not mean this is a crisis dodged for Trump and he is now free from fresh legal and political jeopardy. Quite the reverse.\
RELATED: Trump raised prosecuting Clinton with top White House, Justice officials\
At the very least, the latest developments underline how Trump's senior subordinates may have shielded a President unschooled in constitutional norms from disastrous steps that could have put his presidency in peril.\
And it leaves anyone on the outside wondering what other potential disasters top officials like McGahn, former Attorney General Jeff Sessions and current Deputy Attorney General Rod Rosenstein might have prevented.\
They also raise questions about the capacity of a now-understaffed White House and legal counsel's operation to protect the President from current or future transgressions.\
It will be impossible to confirm, given the habitual silence from the special counsel's office, but the revelations hint at the possibility that Robert Mueller knows much more about what went on in the corridors of the West Wing than has been publicly revealed.\
That will play into rising tensions in Washington amid expectations that the endgame of Mueller's probe is in sight and speculation about possible indictments targeting Trump world and the content of his final report.\
Bombshell reports by CNN and The New York Times about the President's intentions emerged on another surreal day in Washington that saw shocking disclosures about Ivanka Trump's emails and a huge foreign policy pivot over Saudi Arabia."])
s = pd.Series(data)


## Play with LDA


In [0]:
tokens = data_kg_fake_news['text'].astype('U').apply(tokenize2)

In [26]:
print(tokens.__class__)
print(tokens.shape)

<class 'pandas.core.series.Series'>
(12999,)


In [27]:
!pip install -U gensim

from gensim import corpora, models
import gensim

dictionary = corpora.Dictionary(tokens)
print(dictionary.__class__)


Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)
<class 'gensim.corpora.dictionary.Dictionary'>


In [28]:
print(len(dictionary.token2id))
corpus = [dictionary.doc2bow(text) for text in tokens]
print(len(corpus))
print(data_kg_fake_news['text'][123])
print(tokens[123])
print(len(corpus[123]))
print(corpus[123])

122005
12999
Same people all the time , i dont know how you can fix this corruption http://www.fromthewilderness.com/free/ww3/10_09_01_krongard.html
['peopl', 'time', 'dont', 'know', 'fix', 'corrupt', 'http']
7
[(68, 1), (329, 1), (563, 1), (651, 1), (709, 1), (4324, 1), (4846, 1)]


In [0]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=10)


In [30]:
print(ldamodel.print_topics(num_topics=-1, num_words=9))

[(0, '0.005*"peopl" + 0.005*"one" + 0.004*"war" + 0.004*"world" + 0.004*"state" + 0.004*"would" + 0.004*"time" + 0.004*"year" + 0.003*"govern"'), (1, '0.014*"que" + 0.008*"del" + 0.005*"die" + 0.005*"por" + 0.004*"der" + 0.004*"con" + 0.004*"para" + 0.004*"vaccin" + 0.004*"cancer"'), (2, '0.014*"clinton" + 0.013*"trump" + 0.009*"hillari" + 0.008*"elect" + 0.005*"state" + 0.005*"vote" + 0.005*"email" + 0.005*"obama" + 0.005*"said"')]


In [31]:
print(ldamodel[(68, 1), (329, 1), (563, 1), (651, 1), (709, 1), (4324, 1), (4846, 1)])

[(0, 0.04787037), (1, 0.0457006), (2, 0.90642905)]


In [32]:
tokens_test = s.astype('U').apply(tokenize2)
corpus_test = [dictionary.doc2bow(text) for text in tokens_test]
print(ldamodel[corpus_test[0]])

[(0, 0.2268284), (2, 0.7710221)]
